In [1]:
# pip install qiskit==0.31.0

In [ ]:
from get_test_datasets import return_dataset
test_dataset = return_dataset()

Generating QFT circuits
Generating Addition circuits


 62%|███████████████████████████████████████████████████████████████                                      | 6736/10795 [00:32<00:17, 225.96it/s]

In [ ]:
len(test_dataset)

In [ ]:
import tensorflow as tf
from tensorflow import keras

from helper import *

In [ ]:
mlp_edge_feature = tf.keras.models.load_model("models/mlp_edge_features.keras")
dnn = tf.keras.models.load_model("models/dnn.keras")
mlp = tf.keras.models.load_model("models/mlp.keras")

In [4]:

def pred_layout(l, num_qubits):
    layout=[]
    for i in range(len(l[0])):
        layout_i = []
        for slots in l[:num_qubits]:
            if np.argmax(slots[i]) != num_qubits:
                layout_i.append(np.argmax(slots[i]))
            else:
                layout_i.append(np.nan)
        layout.append(layout_i)
    return layout

def controlla_rip(l):
    rip = []
    for i in range(len(l)):
        for j in range(i+1, len(l)):
            if l[i] == l[j]:
                if (i,j) not in rip:
                    rip.append((i,j))

    return rip

def pred_layout_diff_elem(l,num_qubits):
    layout=[]
    for i in range(len(l[0])):
        check = 0
        while check != 1:
            layout_i = []
            for slots in l[:num_qubits]:
                if np.argmax(slots[i]) != num_qubits:
                    layout_i.append(np.argmax(slots[i]))
                else:
                    layout_i.append(np.nan)
            rip = controlla_rip(layout_i)
            if rip == []:
                check = 1
            else:
                for index in rip:
                    if l[index[0]][i][layout_i[index[0]]] > l[index[1]][i][layout_i[index[1]]]:
                        l[index[1]][i][layout_i[index[1]]]=0
                    else:
                        l[index[0]][i][layout_i[index[0]]] = 0

        layout.append(layout_i)
    return layout


In [5]:
def get_labels(y):
    labels = []
    for i in range(len(y)):
        labels.append(np.where(y[i]==np.max(y[i]))[0][0])
    return labels

num_qubits = 7

for qc in test_dataset[8989:8990]:
    data = datetime.datetime.today() - datetime.timedelta(days=random.randint(1,150))
    backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
    backend = backends[np.random.randint(0,3)]
    l = add_line(qc,backend, optimization_level=3, refresh=True, show= False, datatime=data)
    
    d={}
    for i in range(len(l[0])):
        d[str(l[0][i])] = l[1][i]
    df = pd.DataFrame(d, index=[0])
    last_num_qubits = len(df.columns)-num_qubits
    df = clear_dataset(df, 7)
#     x = get_graph_features(df)
    x = df.iloc[:, 2:last_num_qubits].values
    print(x.shape)
#     print(x.shape)
    labels = df.iloc[:, last_num_qubits:].values
    display(df)
#     pred = (mlp_edge_feature.predict(x))
#     predicted=get_labels(np.reshape(pred,(7,7)))
    predicted = dnn.predict(x)
    
    
    pred_or = np.array(pred_layout(predicted, num_qubits))
    pred_nr = np.array(pred_layout_diff_elem(predicted,num_qubits))
    print(pred_or, pred_nr)
    print(predicted, labels[0])    
#     print(predicted==labels[0])

NameError: name 'df' is not defined

In [399]:
import tensorflow as tf
from tensorflow import keras

In [400]:
from Dataset import *

In [435]:
loaded_model = tf.keras.models.load_model("models/mlp_edge_features.keras")

In [436]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 378)]             0         
_________________________________________________________________
dense (Dense)                (None, 189)               71631     
_________________________________________________________________
dense_1 (Dense)              (None, 94)                17860     
_________________________________________________________________
dense_2 (Dense)              (None, 49)                4655      
Total params: 94,146
Trainable params: 94,146
Non-trainable params: 0
_________________________________________________________________
